In [1]:
import pandas as pd 
import numpy as np 
import time


In [2]:
path =""
train_Data = pd.read_csv(path + "source.csv", header=None)


In [3]:
sentence_list = train_Data.iloc[:,1].values


In [4]:
n_gram = 3

In [5]:
import re
def spliteKeyWord(str):
    regex = r"[\u4e00-\ufaff]|[0-9]+|[a-zA-Z]+\'*[a-z]*"
    matches = re.findall(regex, str, re.UNICODE)
    return matches
print(spliteKeyWord('景氣連3綠燈 今年GDP保三有望'))

['景', '氣', '連', '3', '綠', '燈', '今', '年', 'GDP', '保', '三', '有', '望']


## Building  n-gram Inverted Index 

In [6]:
start_time = time.time()
n_gram_store = [] #  A list used to store every n_gram dictionary 
for i in range(n_gram):
    gram_dict = {} # A dictionary used to contain set with n-gram index 
    i = i+1 
    print("Now processing %s gram" %(i))
    #process every sentence in query 
    for sentence_index, sentence in enumerate(sentence_list):
        
        sentence = spliteKeyWord(sentence)
        
        #Start extract n-gram to gram_list:
        for j in range(len(sentence) - (i-1)): # Preventing "index out of range" error
        
            # Concat char to get N-gram index
            gram_index = ''.join(sentence[j:j+i])
            
            if gram_index not in gram_dict:
                gram_dict[gram_index] = set([sentence_index+1]) # index_offset ->plus one 
            else : 
                gram_dict[gram_index].add(sentence_index+1) # index_offset ->plus one
    n_gram_store.append(gram_dict)       
    
print("--- Cost %s seconds to build n-gram inverted index ---" % (time.time() - start_time))


Now processing 1 gram
Now processing 2 gram
Now processing 3 gram
--- Cost 10.432054042816162 seconds to build n-gram inverted index ---


## Build inverted index by query 

In [7]:
query_list = set()
with open(path + "query.txt") as f:
    queries = f.readlines()
    for query in queries:
        line = query.strip("\n").split(" ")
        for i, string in enumerate(line):
            if i %2 == 0:
                query_list.add(string)

In [8]:
start_time = time.time()
query_store = [] 
query_dict = {} 

for query in (query_list):
    #process every sentence in query 
    for sentence_index, sentence in enumerate(sentence_list):
        if query in sentence:
            if query not in query_dict:
                query_dict[query] = set([sentence_index+1]) # index_offset ->plus one 
            else : 
                query_dict[query].add(sentence_index+1) # index_offset ->plus one       
    
print("--- Cost %s seconds to build query inverted index ---" % (time.time() - start_time))


--- Cost 0.30682921409606934 seconds to build query inverted index ---


## Query 

In [9]:
def English(str):
    regex = r"[a-zA-Z]+\'*[a-z]*"
    matches = re.findall(regex, str, re.UNICODE)
    if matches:
        return True
    else:
        return False


### Using n-gram inverted index

In [ ]:
with open(path + "query.txt") as f:
    queries = f.readlines()
    for query in queries:
        line = query.strip("\n").split(" ")
        
        if len(line) == 3:
            query1 = line[0]
            relation = line[1]
            query2 = line[2]
            if English(query1):
                first_set = n_gram_store[0][query1]
            else:
                first_set = n_gram_store[len(query1)-1][query1]

            if English(query2):
                second_set = n_gram_store[0][query2]
            else:
                second_set = n_gram_store[len(query2)-1][query2]

            if relation == 'and':
                print(sorted(list(first_set & second_set)))
            elif relation == 'or':
                print(sorted(list(first_set | second_set)))
            elif relation == 'not':
                print(sorted(list(first_set - second_set)))
        elif len(line)==5:
            pass
                    
         
        

### Using query inverted index


In [ ]:
with open(path + "query.txt") as f:
    queries = f.readlines()
    for query in queries:
        line = query.strip("\n").split(" ")
        
        if len(line) == 3:
            query1 = line[0]
            relation = line[1]
            query2 = line[2]
            first_set = query_dict[query1]
            second_set = query_dict[query2]
            if relation == 'and':
                print(sorted(list(first_set & second_set)))
            elif relation == 'or':
                print(sorted(list(first_set | second_set)))
            elif relation == 'not':
                print(sorted(list(first_set - second_set)))
        elif len(line)==5:
            pass
                    
         
        

In [12]:
query1 = "川普"
query2 = "美國"
relation = "and"

first_set = n_gram_store[len(query1)-1][query1]
second_set = n_gram_store[len(query2)-1][query2]


In [13]:
if relation =="and": 
    print(sorted(list(first_set & second_set)))

[13, 678, 1225, 1905, 2756, 5218, 5839, 8690, 9409, 9640, 13064, 13961, 14156, 14186, 14481, 16770, 17606, 18090, 19914, 20105, 22030, 26636, 28769, 29177, 34604, 34995, 35007, 35463, 36249, 39440, 40672, 41297, 42578, 43556, 47121, 48139, 49384, 51361, 51696, 52696, 53362, 55588, 59223, 60384, 60388, 64906, 65735, 65739, 66178, 66507, 72616, 72838, 74469, 76031, 79419, 79806, 80345, 81268, 83306, 86922, 86944, 88801, 91921, 92423, 95643, 98797]
